In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import mlflow
import mlflow.sklearn
import joblib 


In [2]:
data_path = "D:/My Projects/ride cancellation ml system/data/ncr_ride_bookings.csv"
df = pd.read_csv(data_path)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)
df.head()


,date,time,booking_id,booking_status,customer_id,vehicle_type,pickup_location,drop_location,avg_vtat,avg_ctat,...,reason_for_cancelling_by_customer,cancelled_rides_by_driver,driver_cancellation_reason,incomplete_rides,incomplete_rides_reason,booking_value,ride_distance,driver_ratings,customer_rating,payment_method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [3]:
df.info()
df.isnull().sum()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   date                               150000 non-null  object 
 1   time                               150000 non-null  object 
 2   booking_id                         150000 non-null  object 
 3   booking_status                     150000 non-null  object 
 4   customer_id                        150000 non-null  object 
 5   vehicle_type                       150000 non-null  object 
 6   pickup_location                    150000 non-null  object 
 7   drop_location                      150000 non-null  object 
 8   avg_vtat                           139500 non-null  float64
 9   avg_ctat                           102000 non-null  float64
 10  cancelled_rides_by_customer        10500 non-null   float64
 11  reason_for_cancelling_by_customer  1050

,avg_vtat,avg_ctat,cancelled_rides_by_customer,cancelled_rides_by_driver,incomplete_rides,booking_value,ride_distance,driver_ratings,customer_rating
count,139500.000000,102000.000000,10500.0,27000.0,9000.0,102000.000000,102000.000000,93000.000000,93000.000000
mean,8.456352,29.149636,1.0,1.0,1.0,508.295912,24.637012,4.230992,4.404584
std,3.773564,8.902577,0.0,0.0,0.0,395.805774,14.002138,0.436871,0.437819
min,2.000000,10.000000,1.0,1.0,1.0,50.000000,1.000000,3.000000,3.000000
25%,5.300000,21.600000,1.0,1.0,1.0,234.000000,12.460000,4.100000,4.200000
50%,8.300000,28.800000,1.0,1.0,1.0,414.000000,23.720000,4.300000,4.500000
75%,11.300000,36.800000,1.0,1.0,1.0,689.000000,36.820000,4.600000,4.800000
max,20.000000,45.000000,1.0,1.0,1.0,4277.000000,50.000000,5.000000,5.000000


In [4]:
required_cols = ["booking_status", "time"]
missing_cols = [c for c in required_cols if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing required columns: {missing_cols}")

df = df.dropna(subset=required_cols).copy()
df.shape


(150000, 21)

In [5]:
df["is_cancelled"] = (
    df["booking_status"]
    .astype(str)
    .str.strip()
    .str.contains("cancelled", case=False, na=False)
    .astype(int)
)

df["is_cancelled"].value_counts()


is_cancelled
0    112500
1     37500
Name: count, dtype: int64

In [6]:
time_col = "booking_time" if "booking_time" in df.columns else "time"
if time_col not in df.columns:
    raise KeyError(f"Time column not found. Available columns: {list(df.columns)}")

df["booking_hour"] = pd.to_datetime(df[time_col], format="%H:%M:%S", errors="coerce").dt.hour
df = df.drop(columns=[c for c in [time_col, "booking_status"] if c in df.columns])
df.head()


,date,booking_id,customer_id,vehicle_type,pickup_location,drop_location,avg_vtat,avg_ctat,cancelled_rides_by_customer,reason_for_cancelling_by_customer,...,driver_cancellation_reason,incomplete_rides,incomplete_rides_reason,booking_value,ride_distance,driver_ratings,customer_rating,payment_method,is_cancelled,booking_hour
0,2024-03-23,"""CNR5884300""","""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,12
1,2024-11-29,"""CNR1326809""","""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,NaN,NaN,...,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI,0,18
2,2024-08-23,"""CNR8494506""","""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,NaN,NaN,...,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card,0,8
3,2024-10-21,"""CNR8906825""","""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,NaN,NaN,...,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI,0,17
4,2024-09-16,"""CNR1950162""","""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,NaN,NaN,...,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI,0,22


In [7]:
drop_cols = ["booking_id", "customer_id"]
X = df.drop(columns=[c for c in ["is_cancelled", *drop_cols] if c in df.columns])
y = df["is_cancelled"]
X.shape, y.shape


((150000, 18), (150000,))

In [8]:
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
len(categorical_cols), len(numerical_cols)


(8, 10)

In [9]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_cols),
        ("num", numerical_transformer, numerical_cols)
    ]
)


In [10]:
model = RandomForestClassifier(
    n_estimators=80,
    max_depth=12,
    min_samples_leaf=5,
    n_jobs=-1,
    random_state=42
)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", model)
])


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

In [12]:
preds = pipeline.predict(X_test)

print("Accuracy:", round(accuracy_score(y_test, preds), 4))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))


Accuracy: 0.9468
              precision    recall  f1-score   support

           0       0.93      1.00      0.97     22500
           1       1.00      0.79      0.88      7500

    accuracy                           0.95     30000
   macro avg       0.97      0.89      0.92     30000
weighted avg       0.95      0.95      0.94     30000

[[22500     0]
 [ 1596  5904]]


False Negative → Driver reaches but customer cancels

False Positive → System wrongly predicts cancellation

In [15]:
import sklearn, skops

if not hasattr(pipeline.named_steps["classifier"], "classes_"):
    pipeline.fit(X_train, y_train)

mlflow.set_tracking_uri("file///D:/My Projects/ride cancellation ml system/mlruns")
mlflow.set_experiment("ride-cancellation")

pip_reqs = [
    f"scikit-learn=={sklearn.__version__}",
    f"skops=={skops.__version__}"
]

with mlflow.start_run():
    mlflow.log_param("model", "RandomForest")
    mlflow.log_metric("accuracy", pipeline.score(X_test, y_test))
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        name="model",
        serialization_format="skops",
        skops_trusted_types=["numpy.dtype"],
        pip_requirements=pip_reqs
    )


d:\My Projects\ride cancellation ml system\venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


In [16]:
os.makedirs("D:/My Projects/ride cancellation ml system/models", exist_ok=True)
joblib.dump(pipeline, "D:/My Projects/ride cancellation ml system/models/model.pkl")


['D:/My Projects/ride cancellation ml system/models/model.pkl']